In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
file_path = 'C:/Users/BERKAN/Desktop/IT/Data Science/DataProjeler/CustomerChurn/CustomerChurnData.csv'
df = pd.read_csv(file_path)
import warnings

warnings.filterwarnings('ignore')
sns.set_style()

df = df.drop(columns=['customerID'])
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')
df = df.drop_duplicates()
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7010 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7010 non-null   object 
 1   SeniorCitizen     7010 non-null   object 
 2   Partner           7010 non-null   object 
 3   Dependents        7010 non-null   object 
 4   tenure            7010 non-null   int64  
 5   PhoneService      7010 non-null   object 
 6   MultipleLines     7010 non-null   object 
 7   InternetService   7010 non-null   object 
 8   OnlineSecurity    7010 non-null   object 
 9   OnlineBackup      7010 non-null   object 
 10  DeviceProtection  7010 non-null   object 
 11  TechSupport       7010 non-null   object 
 12  StreamingTV       7010 non-null   object 
 13  StreamingMovies   7010 non-null   object 
 14  Contract          7010 non-null   object 
 15  PaperlessBilling  7010 non-null   object 
 16  PaymentMethod     7010 non-null   object 
 17  

In [21]:


import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE


categorical_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)


scaler = StandardScaler()
df_encoded[['MonthlyCharges', 'TotalCharges', 'tenure']] = scaler.fit_transform(df_encoded[['MonthlyCharges', 'TotalCharges', 'tenure']])


le = LabelEncoder()
df_encoded['Churn'] = le.fit_transform(df_encoded['Churn']) 


X = df_encoded.drop('Churn', axis=1) 
y = df_encoded['Churn']  


smote = SMOTE(sampling_strategy='minority', random_state=42)
X_res, y_res = smote.fit_resample(X, y)


print(f"Orijinal veri (Churn): {y.value_counts()}")
print(f"SMOTE sonrası veri (Churn): {y_res.value_counts()}")



Orijinal veri (Churn): Churn
0    5153
1    1857
Name: count, dtype: int64
SMOTE sonrası veri (Churn): Churn
0    5153
1    5153
Name: count, dtype: int64


In [23]:

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)


model = xgb.XGBClassifier(random_state=42)


model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8370514064015518

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.81      0.83      1036
           1       0.82      0.86      0.84      1026

    accuracy                           0.84      2062
   macro avg       0.84      0.84      0.84      2062
weighted avg       0.84      0.84      0.84      2062



In [25]:

y_pred = model.predict(X_test)


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


precision = precision_score(y_test, y_pred)
print("Precision:", precision)


recall = recall_score(y_test, y_pred)
print("Recall:", recall)


f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)


print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.8370514064015518
Precision: 0.8182656826568265
Recall: 0.8645224171539961
F1-Score: 0.8407582938388625

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.81      0.83      1036
           1       0.82      0.86      0.84      1026

    accuracy                           0.84      2062
   macro avg       0.84      0.84      0.84      2062
weighted avg       0.84      0.84      0.84      2062



In [27]:

y_train_pred = model.predict(X_train)


y_test_pred = model.predict(X_test)


train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_accuracy)


test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)


print("\nClassification Report on Training Data:")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


if train_accuracy > test_accuracy:
    print("\nModel overfitting olabilir çünkü eğitim doğruluğu test doğruluğundan daha yüksek.")
else:
    print("\nModelin doğruluğu eğitim ve test verisinde benzer olduğu için overfitting gözükmüyor.")


Training Accuracy: 0.9338913148956817
Test Accuracy: 0.8370514064015518

Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      4117
           1       0.92      0.96      0.94      4127

    accuracy                           0.93      8244
   macro avg       0.93      0.93      0.93      8244
weighted avg       0.93      0.93      0.93      8244


Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.86      0.81      0.83      1036
           1       0.82      0.86      0.84      1026

    accuracy                           0.84      2062
   macro avg       0.84      0.84      0.84      2062
weighted avg       0.84      0.84      0.84      2062


Model overfitting olabilir çünkü eğitim doğruluğu test doğruluğundan daha yüksek.


In [29]:

model = xgb.XGBClassifier(
    random_state=42,
    max_depth=3,  # Derinliği sınırlama
    n_estimators=100,  # Daha az sayıda ağaç
    learning_rate=0.1,
    subsample=0.8,  # Veri örnekleme oranını azaltma
    colsample_bytree=0.8,  # Özellik örnekleme oranını azaltma
    reg_lambda=1,  # L2 düzenlileştirme
    reg_alpha=0.1,  # L1 düzenlileştirme
    gamma=0.1  # Daha sıkı bir ağaç büyümesi
)


model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")


Training Accuracy: 0.8096797671033479
Test Accuracy: 0.7982541222114452


In [33]:
from sklearn.metrics import accuracy_score, classification_report


y_train_pred = model.predict(X_train)


y_test_pred = model.predict(X_test)


train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy}")


test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")


print("\nClassification Report on Training Data:")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))




Training Accuracy: 0.8096797671033479
Test Accuracy: 0.7982541222114452

Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.84      0.77      0.80      4117
           1       0.79      0.85      0.82      4127

    accuracy                           0.81      8244
   macro avg       0.81      0.81      0.81      8244
weighted avg       0.81      0.81      0.81      8244


Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.83      0.76      0.79      1036
           1       0.77      0.84      0.81      1026

    accuracy                           0.80      2062
   macro avg       0.80      0.80      0.80      2062
weighted avg       0.80      0.80      0.80      2062

